In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 100)

In [2]:
Cases = pd.read_csv('data/Cases2018.csv', encoding='iso-8859-1')

### Examine Data, drop label and related columns, figure out NaNs

In [3]:
unneeded =Cases[['caseId','docketId','lexisCite','term','naturalCourt','docket','chief','caseName','caseIssuesId','voteId','dateDecision', 'decisionType','usCite','sctCite','ledCite', 'issue','issueArea','decisionDirection','decisionDirectionDissent','authorityDecision1','authorityDecision2','lawType','lawSupp','lawMinor','majOpinWriter','majOpinAssigner']]
labels =Cases[['declarationUncon', 'caseDisposition', 'caseDispositionUnusual','partyWinning', 'precedentAlteration','voteUnclear','splitVote','majVotes','minVotes']]

In [4]:
Cases.drop(unneeded, axis=1, inplace=True)
Cases.drop(labels, axis=1, inplace=True)

In [5]:
Cases.head()

dateArgument   dateRearg  petitioner  petitionerState  respondent  \
0     1/9/1946  10/23/1946         198              NaN       172.0   
1   10/10/1945  10/17/1946         100              NaN        27.0   
2    11/8/1945  10/18/1946         209              NaN        27.0   
3    1/31/1946  10/25/1946          27              NaN       170.0   
4   10/25/1946         NaN          27              NaN       176.0   

   respondentState  jurisdiction  adminAction  adminActionState  \
0              NaN             6          NaN               NaN   
1              NaN             1          NaN               NaN   
2              NaN             2         66.0               NaN   
3              NaN             1         67.0               NaN   
4              NaN             1          NaN               NaN   

   threeJudgeFdc  caseOrigin  caseOriginState  caseSource  caseSourceState  \
0            0.0        51.0              6.0        29.0              NaN   
1            0.0       123.0             52.0        30.0              NaN   
2            1.0       107.0             42.0       107.0             42.0   
3            0.0         3.0              NaN         3.0              NaN   
4            0.0         3.0              NaN         3.0              NaN   

   lcDisagreement  certReason  lcDisposition  lcDispositionDirection  
0             0.0        11.0            2.0                     1.0  
1             0.0         4.0            2.0                     1.0  
2             0.0         1.0            NaN                     2.0  
3             0.0        10.0            NaN                     2.0  
4             0.0         2.0            NaN                     2.0

In [6]:
Cases.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8893 entries, 0 to 8892
Data columns (total 18 columns):
dateArgument              7858 non-null object
dateRearg                 177 non-null object
petitioner                8893 non-null int64
petitionerState           1814 non-null float64
respondent                8892 non-null float64
respondentState           2492 non-null float64
jurisdiction              8893 non-null int64
adminAction               2455 non-null float64
adminActionState          663 non-null float64
threeJudgeFdc             8884 non-null float64
caseOrigin                8549 non-null float64
caseOriginState           2461 non-null float64
caseSource                8692 non-null float64
caseSourceState           2133 non-null float64
lcDisagreement            8884 non-null float64
certReason                8807 non-null float64
lcDisposition             7641 non-null float64
lcDispositionDirection    8685 non-null float64
dtypes: float64(14), int64(2), object

##### Dates not important, just the fact oral arguments occured or not

In [7]:
Cases[['dateArgument','dateRearg']]=Cases[['dateArgument', 'dateRearg']].notnull().astype(int)

##### filling some NaN in petitioner/respondent state columns with 0 a variable to mean unspecified

In [8]:
Cases.groupby('respondent').count()

dateArgument  dateRearg  petitioner  petitionerState  \
respondent                                                         
1.0                   41         41          41                5   
2.0                    6          6           6                2   
3.0                  180        180         180               28   
4.0                  141        141         141               15   
5.0                   72         72          72                8   
6.0                   48         48          48                3   
7.0                   67         67          67               18   
8.0                   93         93          93               42   
9.0                   19         19          19               17   
10.0                  12         12          12                8   
11.0                   2          2           2                2   
13.0                   9          9           9                5   
14.0                   2          2           2                0   
15.0                  10         10          10                2   
16.0                  52         52          52                6   
17.0                   4          4           4                0   
18.0                  11         11          11                3   
19.0                 223        223         223               34   
20.0                  24         24          24                0   
21.0                  86         86          86               19   
23.0                   6          6           6                0   
24.0                   9          9           9                0   
25.0                  25         25          25               19   
26.0                  24         24          24                8   
27.0                1056       1056        1056               76   
28.0                1394       1394        1394              141   
100.0                387        387         387              208   
101.0                  3          3           3                1   
102.0                 22         22          22                0   
103.0                  1          1           1                0   
...                  ...        ...         ...              ...   
361.0                  2          2           2                1   
362.0                 13         13          13                2   
363.0                 13         13          13                1   
364.0                  2          2           2                0   
366.0                 12         12          12                1   
368.0                 52         52          52                0   
369.0                 68         68          68                2   
370.0                  1          1           1                1   
371.0                 20         20          20                2   
374.0                  1          1           1                1   
377.0                  5          5           5                1   
382.0                 66         66          66                0   
384.0                  1          1           1                0   
386.0                  1          1           1                0   
389.0                  1          1           1                0   
390.0                  1          1           1                1   
392.0                  2          2           2                0   
393.0                  4          4           4                0   
394.0                  3          3           3                0   
396.0                  2          2           2                0   
397.0                  1          1           1                0   
403.0                  5          5           5                0   
405.0                 12         12          12                0   
406.0                  5          5           5                0   
407.0                  4          4           4                0   
408.0                  8          8           8                3   


In [9]:
Cases[['petitionerState','adminActionState','threeJudgeFdc','respondentState',]] = Cases[['petitionerState','adminActionState','threeJudgeFdc','respondentState',]].fillna(value=0.0)
Cases['respondent'] = Cases[['respondent']].fillna(value=501)# There was already a code for unidentfiable which i reused for NaN here
Cases['adminAction'] = Cases[['adminAction']].fillna(value=118.0)# There was already a code for unidentfiable which i reused for NaN here
Cases['caseOrigin'] = Cases[['caseOrigin']].fillna(value=0.0)#meaning originated in supreme court
Cases['caseSource'] = Cases[['caseSource']].fillna(value=0.0)##meaning originated in supreme court
Cases['caseOriginState'] = Cases[['caseOriginState']].fillna(value=0.0)
Cases['caseSourceState'] = Cases[['caseSourceState']].fillna(value=0.0)
Cases['lcDisagreement'] = Cases[['lcDisagreement']].fillna(value=0.0)
Cases['certReason'] = Cases[['certReason']].fillna(value=12)
Cases['lcDisposition'] = Cases[['lcDisposition']].fillna(value=0.0)
Cases['lcDispositionDirection'] = Cases[['lcDispositionDirection']].fillna(value=3.0)

dateArgument  dateRearg  petitioner  petitionerState  respondent  \
0             1          1         198              0.0       172.0   
1             1          1         100              0.0        27.0   
2             1          1         209              0.0        27.0   
3             1          1          27              0.0       170.0   
4             1          0          27              0.0       176.0   

   respondentState  jurisdiction  adminAction  adminActionState  \
0              0.0             6        118.0               0.0   
1              0.0             1        118.0               0.0   
2              0.0             2         66.0               0.0   
3              0.0             1         67.0               0.0   
4              0.0             1        118.0               0.0   

   threeJudgeFdc  caseOrigin  caseOriginState  caseSource  caseSourceState  \
0            0.0        51.0              6.0        29.0              0.0   
1            0.0       123.0             52.0        30.0              0.0   
2            1.0       107.0             42.0       107.0             42.0   
3            0.0         3.0              0.0         3.0              0.0   
4            0.0         3.0              0.0         3.0              0.0   

   lcDisagreement  certReason  lcDisposition  lcDispositionDirection  
0             0.0        11.0            2.0                     1.0  
1             0.0         4.0            2.0                     1.0  
2             0.0         1.0            0.0                     2.0  
3             0.0        10.0            0.0                     2.0  
4             0.0         2.0            0.0                     2.0

In [16]:
labels.groupby('partyWinning').count()

declarationUncon  caseDisposition  caseDispositionUnusual  \
partyWinning                                                              
0.0                       3194             3142                    3194   
1.0                       5679             5600                    5679   
2.0                          3                2                       3   

              precedentAlteration  voteUnclear  splitVote  majVotes  minVotes  
partyWinning                                                                   
0.0                          3194         3194       3195      3195      3195  
1.0                          5679         5679       5679      5679      5679  
2.0                             3            3          3         3         3

In [18]:
labels.head()

declarationUncon  caseDisposition  caseDispositionUnusual  partyWinning  \
0               1.0              3.0                     0.0           1.0   
1               1.0              2.0                     0.0           0.0   
2               1.0              2.0                     0.0           0.0   
3               1.0              2.0                     0.0           0.0   
4               1.0              3.0                     0.0           1.0   

   precedentAlteration  voteUnclear  splitVote  majVotes  minVotes  
0                  1.0          0.0          1         8         1  
1                  0.0          0.0          1         6         3  
2                  0.0          0.0          1         5         4  
3                  0.0          0.0          1         5         3  
4                  0.0          0.0          1         6         3

In [19]:
## 0 = petitioner loses 1 = petitioner wins

In [20]:
finalLabels = labels['partyWinning']

In [22]:
Cases['result'] = labels['partyWinning']

In [23]:
Cases.groupby('result')

dateArgument  dateRearg  petitioner  petitionerState  respondent  \
0             1          1         198              0.0       172.0   
1             1          1         100              0.0        27.0   
2             1          1         209              0.0        27.0   
3             1          1          27              0.0       170.0   
4             1          0          27              0.0       176.0   

   respondentState  jurisdiction  adminAction  adminActionState  \
0              0.0             6        118.0               0.0   
1              0.0             1        118.0               0.0   
2              0.0             2         66.0               0.0   
3              0.0             1         67.0               0.0   
4              0.0             1        118.0               0.0   

   threeJudgeFdc  caseOrigin  caseOriginState  caseSource  caseSourceState  \
0            0.0        51.0              6.0        29.0              0.0   
1            0.0       123.0             52.0        30.0              0.0   
2            1.0       107.0             42.0       107.0             42.0   
3            0.0         3.0              0.0         3.0              0.0   
4            0.0         3.0              0.0         3.0              0.0   

   lcDisagreement  certReason  lcDisposition  lcDispositionDirection  result  
0             0.0        11.0            2.0                     1.0     1.0  
1             0.0         4.0            2.0                     1.0     0.0  
2             0.0         1.0            0.0                     2.0     0.0  
3             0.0        10.0            0.0                     2.0     0.0  
4             0.0         2.0            0.0                     2.0     1.0